In [6]:
# %% Imports & constants
from __future__ import annotations

import os, json
from functools import lru_cache
from io import StringIO
from typing import Optional, TypedDict

import numpy as np
import pandas as pd
import ipywidgets as widgets
import plotly.graph_objects as go
from plotly.graph_objs import FigureWidget

import requests
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

from IPython.display import Javascript, display, HTML

# ────────────────────────────────────────────────────────────────────────────────
# Visual CSS
display(HTML("""
<style>
:root{
  --bg:#05060c; --panel:#0b0f18; --grid:rgba(255,255,255,.08);
  --c1:#37f3ff; --ok:#00ffb2; --warn:#ff58c6; --ink:#eaf2ff;
}
.jp-Notebook, body.jp-Notebook {
  background:
    radial-gradient(1200px 700px at 20% -10%, rgba(55,243,255,.12), transparent 60%),
    radial-gradient(900px 600px at 85% -5%, rgba(184,140,255,.10), transparent 60%),
    linear-gradient(180deg, #05060c 0%, #070b13 100%) !important;
}
.widget-label, .widget-inline-hbox label, .widget-dropdown label, .widget-toggle-button label {
  color: var(--ink) !important;
}
.widget-hbox, .widget-inline-hbox, .widget-box, .widget-output {
  border-radius: 14px !important;
}
</style>
"""))

# ────────────────────────────────────────────────────────────────────────────────
# Config

SEC_API_KEY = os.environ.get("SEC_API_KEY", "ca91a88f0a033a9774da5fba074cb8660f145f5ce5f3eb8eac83973c458f7923")
if not SEC_API_KEY:
    raise RuntimeError("🔑 Please set SEC_API_KEY in your environment.")

STATUS_COLORS = {"Alive": "#00ffb2", "Called": "#ff58c6"}

WINDOW_NAME = "miniFiling"   # named popup window we reuse

# ────────────────────────────────────────────────────────────────────────────────
# Requests session with retries
def get_session() -> requests.Session:
    s = requests.Session()
    retries = Retry(
        total=5,
        backoff_factor=0.8,
        status_forcelist=(429, 500, 502, 503, 504),
        allowed_methods=("POST",),
        raise_on_status=False,
        respect_retry_after_header=True,
    )
    s.mount("https://", HTTPAdapter(max_retries=retries))
    s.headers.update({"User-Agent": "cyn-lifespan-voila/1.0"})
    return s

SESSION = get_session()

# ────────────────────────────────────────────────────────────────────────────────
# Front-end hook:
#  - Opens a centered mini window DURING the Plotly click (user gesture kept)
#  - Saves a handle at window.__miniFilingWin for later navigation
def register_mini_popup(*, name=WINDOW_NAME, width=1000, height=800):
    js = r"""
    (function() {
      function loaderHtml() {
        return `<!doctype html><title>Loading…</title>
<body style="margin:0;display:grid;place-items:center;background:#0b0f18;color:#eaf2ff;font:16px ui-monospace;">Loading…</body>`;
      }
      function attach() {
        var plots = document.querySelectorAll('.js-plotly-plot');
        plots.forEach(function(gd){
          if (gd._miniHooked) return;
          gd._miniHooked = true;

          gd.on('plotly_click', function(){
            var sw = window.innerWidth  || document.documentElement.clientWidth  || screen.width;
            var sh = window.innerHeight || document.documentElement.clientHeight || screen.height;
            var left = Math.max(0, (sw - __W__)/2);
            var top  = Math.max(0, (sh - __H__)/2);
            var features = [
              "toolbar=no","location=no","status=no","menubar=no",
              "scrollbars=yes","resizable=yes",
              "width=__W__","height=__H__","left="+left,"top="+top
            ].join(",");
            var win = window.open('about:blank', __NAME__, features);
            if (win) {
              try { win.document.open(); win.document.write(loaderHtml()); win.document.close(); } catch (e) {}
              try { win.focus(); } catch (e) {}
              // Save a direct handle for later navigation from Python-side JS
              try { window.__miniFilingWin = win; } catch(e) {}
            }
          });
        });
      }
      if (document.querySelector('.js-plotly-plot')) attach();
      setTimeout(attach, 200);
      setTimeout(attach, 800);
    })();
    """
    js = (js.replace("__W__", str(width))
            .replace("__H__", str(height))
            .replace("__NAME__", json.dumps(name)))
    display(Javascript(js))

# ────────────────────────────────────────────────────────────────────────────────
# Full-text search helper

class FilingHit(TypedDict, total=False):
    filedAt: str
    filingDate: str
    linkToFilingDetails: str
    filingUrl: str
    url: str

@lru_cache(maxsize=512)
def full_text_search(api_key: str, cusip: str) -> Optional[str]:
    endpoint = "https://api.sec-api.io/full-text-search"
    payload = {
        "query": f'"{cusip}"',
        "formTypes": ["424B2"],
        "startDate": "2020-01-01",
        "endDate": "2026-06-30",
        "page": "1",
    }
    try:
        r = SESSION.post(
            f"{endpoint}?token={api_key}",
            json=payload,
            headers={"Authorization": api_key},
            timeout=15,
        )
    except requests.RequestException:
        return None

    if r.status_code >= 400:
        return None

    try:
        data = r.json()
    except ValueError:
        return None

    hits = data.get("filings") or data.get("data", {}).get("filings", [])
    if not hits:
        return None

    def filed_dt(hit: FilingHit) -> str:
        return hit.get("filedAt") or hit.get("filingDate", "") or ""

    best: FilingHit = max(hits, key=filed_dt)
    return best.get("linkToFilingDetails") or best.get("filingUrl") or best.get("url")

# ────────────────────────────────────────────────────────────────────────────────
# Data

CSV = """CUSIP,Start,End,Status,Lifespan,Yield,Issuer
09711A2C1,2023-06-02,2024-02-07,Called,8.33,-,BAML
90279GDY2,2023-04-19,2024-01-24,Called,9.33,-,UBS
09711GQP7,2023-06-22,2023-12-28,Called,6.30,-,UBS
48133VEF3,2023-03-24,2024-03-28,Called,12.33,-,JPM
17331CPP6,2023-02-17,2023-08-22,Called,6.20,-,Citi
90279F7L9,2023-03-24,2023-12-29,Called,9.33,-,UBS
06745MRA8,2023-07-12,2024-05-20,Called,10.43,-,Barc
90279GYZ6,2023-07-25,2024-01-30,Called,6.30,-,UBS
90279GZM4,2023-07-25,2024-05-31,Called,10.37,-,UBS
06745N3K0,2023-08-09,2024-02-16,Called,6.37,-,Barc
09711AHL5,2023-08-21,2024-01-25,Called,5.23,-,BAML
90279WDH4,2023-09-15,2023-12-15,Called,3.03,-,UBS
09711ARD2,2023-09-26,2023-12-29,Called,3.13,-,BAML
09711ARJ9,2023-09-29,2024-07-05,Called,9.33,-,BofA
17291QVB7,2023-10-10,2024-04-15,Called,6.27,-,Citi
09711AWA2,2023-10-18,2024-01-23,Called,3.23,-,BAML
61775MLF1,2023-10-27,2024-02-01,Called,3.23,-,MS
61775MNX0,2023-11-01,2024-02-06,Called,3.23,-,MS
09710P2V7,2023-11-03,2024-05-08,Called,6.23,-,BAML
06745PAU5,2023-12-11,2024-03-18,Called,3.27,-,Barc
90279WC77,2024-01-02,2024-07-08,Called,6.27,-,UBS
90279WF66,2024-01-10,2024-07-15,Called,6.23,-,UBS
09710PL61,2024-01-16,2024-04-19,Called,3.13,-,BAML
61771WRJ9,2024-01-25,2025-01-30,Called,12.37,9,MS
90279WT61,2024-02-08,2024-11-14,Called,9.33,10.75,UBS
09710PXC5,2024-02-16,2024-12-17,Called,10.17,10.1,BofA
40057YEG4,2024-02-23,2024-05-29,Called,3.20,-,GS
05612CPG1,2024-03-05,2024-09-16,Called,6.50,-,BNP
40057YPZ0,2024-03-18,2024-06-24,Called,3.27,-,GS
90307DAH5,2024-04-09,2024-11-14,Called,7.30,10.65,UBS
09711BNH5,2024-04-17,2024-10-22,Called,6.27,10.8,BofA
06745QMN6,2024-04-26,2024-08-02,Called,3.27,-,Barc
09711BZC3,2024-05-13,2024-12-18,Called,7.30,9.7,BofA
90307DPP1,2024-05-22,,Alive,13.73,10.15,UBS
09711DQE5,2024-06-05,2024-09-10,Called,3.23,-,BofA
06745U6U9,2024-06-14,2024-11-21,Called,5.33,-,Barc
90307D2A9,2024-07-05,,Alive,12.27,10,UBS
09711DDE9,2024-07-17,2024-11-21,Called,4.23,-,BofA
40058ED52,2024-07-25,2024-11-29,Called,4.23,-,GS
09711DHM7,2024-08-02,2024-12-05,Called,4.17,-,BofA
61776MZ73,2024-08-07,2025-02-12,Called,6.30,-,MS
06745UQ86,2024-08-12,2024-11-19,Called,3.30,-,Barc
05613FP99,2024-08-15,2024-11-20,Called,3.23,-,BNP
90307QCV3,2024-09-09,2025-02-13,Called,5.23,-,UBS
40058F3S0,2024-09-12,2024-12-17,Called,3.20,-,GS
09711FXV4,2024-09-17,2025-02-21,Called,5.23,-,BofA
61776R5H3,2024-10-04,,Alive,9.23,-,MS
06745YB76,2024-10-10,,Alive,9.03,10.3,Barc
06745YCA8,2024-10-18,,Alive,8.77,10.3,Barc
05613LLU3,2024-10-23,,Alive,8.60,10.56,BNP
61776WKU6,2024-10-25,,Alive,8.53,10.15,MS
05613LTV3,2024-10-28,,Alive,8.43,10.56,BNP
61776WH55,2024-11-19,,Alive,7.70,10,MS
06745YP97,2024-11-25,,Alive,7.50,10,Barc
09711F6T9,2024-11-29,,Alive,7.37,10,BofA
90307QZA4,2024-12-09,,Alive,7.03,10,UBS
05614BPS5,2024-12-18,,Alive,6.73,10.25,BNP
61777RRD7,2024-12-23,,Alive,6.57,11.6,MS
65541KAR5,2025-01-13,2025-07-08,Alive,5.87,10.4,NOM
05615G7H7,2025-01-16,,Alive,5.77,10.44,BNP
09711GBT1,2025-02-04,,Alive,5.13,10,BofA
40058GSB8,2025-02-07,,Alive,5.03,10,GS
90308QHE5,2025-02-14,,Alive,4.80,10,UBS
05615GZX1,2025-02-27,,Alive,4.37,10,BNP
09711GUW3,2025-03-05,,Alive,4.17,10.05,BofA
06746B2X8,2025-03-11,2025-06-18,Called,3.30,11,Barc
90308VGL9,2025-03-14,,Alive,3.87,10.7,UBS
61778CZ73,2025-03-19,,Alive,3.70,10.5,MS
05615J2W3,2025-04-02,2025-07-07,Called,3.20,10,BNP
40058HLF4,2025-04-10,2025-06-13,Called,2.13,10.7,GS
06746BHL8,2025-04-14,,Alive,2.83,11,Barc
09711GTH8,2025-04-16,,Alive,2.77,10,BofA
61778JWH9,2025-04-23,,Alive,2.53,12.15,MS
17333JL32,2025-04-28,,Alive,2.37,11,Citi
17333J3G3,2025-05-05,,Alive,2.13,12,Citi
09711HLH4,2025-05-27,,Alive,1.40,11,BofA
90308VQ53,2025-06-03,,Alive,1.17,11,UBS
40058JEJ0,2025-06-13,,Alive,0.83,10.55,GS
06746C6S3,2025-06-17,,Alive,0.70,11.2,Barc
90308V5R8,2025-06-27,,Alive,0.37,10,UBS
"""

df = pd.read_csv(StringIO(CSV), parse_dates=["Start", "End"])
df["Start_str"] = df["Start"].dt.strftime("%Y-%m-%d")
df["End_str"] = pd.to_datetime(df["End"], errors="coerce").dt.strftime("%Y-%m-%d").fillna("—")
df["Status"] = pd.Categorical(df["Status"], categories=["Alive", "Called"], ordered=True)

# ────────────────────────────────────────────────────────────────────────────────
# Widgets

status_dd = widgets.Dropdown(
    options=["All"] + list(df["Status"].cat.categories),
    value="All",
    description="Status:",
)
toggle_avg = widgets.ToggleButton(
    value=False,
    description="Show Average",
    button_style="info",
)
reset_btn  = widgets.Button(
    description="Reset",
    tooltip="Reset chart view and controls",
)
output = widgets.Output(layout={"border": "1px solid rgba(121,168,255,.22)", "padding": "10px", "border_radius": "14px"})

# ────────────────────────────────────────────────────────────────────────────────
# State holder
class UIState:
    def __init__(self):
        self.fw: Optional[FigureWidget] = None
        self.orig_sizes: list[float] = []
        self.last_idx: Optional[int] = None

state = UIState()

# ────────────────────────────────────────────────────────────────────────────────
# Update figure

def update_figure(*_):
    st = status_dd.value
    d = df if st == "All" else df[df["Status"] == st]

    xvals = d["Start"].to_list()
    yvals = d["Lifespan"].astype(float).to_list()
    cd    = d[["CUSIP","Start_str","End_str","Status","Lifespan","Yield","Issuer"]].values

    base_size = 12.0
    sizes = [base_size] * len(d)

    scatter = go.Scatter(
        x=xvals, y=yvals, mode="markers",
        marker=dict(
            size=sizes,
            color=[STATUS_COLORS.get(s, "gray") for s in d["Status"]],
            line=dict(width=1.2, color="#eaf2ff")
        ),
        customdata=cd,
        hovertemplate=(
            "<b style='color:#37f3ff'>CUSIP</b> %{customdata[0]}<br>"
            "<b>Start:</b> %{customdata[1]}<br>"
            "<b>End:</b> %{customdata[2]}<br>"
            "<b>Status:</b> %{customdata[3]}<br>"
            "<b>Lifespan:</b> %{y:.2f} mo<br>"
            "<b>Yield:</b> %{customdata[5]}<br>"
            "<b>Issuer:</b> %{customdata[6]}<extra></extra>"
        ),
        selected=dict(marker=dict(opacity=1)),
        unselected=dict(marker=dict(opacity=1)),
    )

    fw = FigureWidget(scatter)
    fw.update_layout(
        title="CYN Lifespan vs. Start Date",
        template="plotly_dark",
        font=dict(color="#eaf2ff"),
        xaxis_title="Start Date",
        yaxis_title="Lifespan (months)",
        clickmode="event+select",
        xaxis=dict(type="date", gridcolor="rgba(255,255,255,.08)"),
        yaxis=dict(gridcolor="rgba(255,255,255,.08)"),
        width=900, height=600,
        margin=dict(l=60, r=40, t=60, b=50),
        paper_bgcolor="rgba(0,0,0,0)",
        plot_bgcolor="rgba(12,16,26,0.55)",
    )

    # frame
    fw.add_shape(type="rect", xref="paper", yref="paper",
                 x0=0, y0=0, x1=1, y1=1,
                 line=dict(color="#37f3ff", width=1), opacity=0.25)

    # avg line
    fw.layout.shapes = tuple([s for s in fw.layout.shapes])
    fw.layout.annotations = ()
    if toggle_avg.value and yvals:
        avg = float(np.mean(yvals))
        fw.add_hline(
            y=avg, line_dash="dash", line_color="#ffd166",
            annotation_text=f"Average: {avg:.2f} mo", annotation_position="top left", opacity=0.95
        )

    # state + handlers
    state.fw = fw
    state.orig_sizes = sizes.copy()
    state.last_idx = None
    fw.data[0].on_click(on_scatter_click)

    with output:
        output.clear_output(wait=True)
        display(fw)
        # ensure mini popup opens during the front-end click and we keep a handle
        register_mini_popup(name=WINDOW_NAME, width=1000, height=800)

# ────────────────────────────────────────────────────────────────────────────────
# Click handler

def on_scatter_click(trace, points, _state):
    if not points.point_inds:
        return
    idx = points.point_inds[0]

    if state.last_idx == idx:
        state.last_idx = None
        trace.marker.size = state.orig_sizes
        trace.figure.layout.annotations = ()
        return

    state.last_idx = idx
    cusip, start_s, end_s, status, lifespan, yld, issuer = trace.customdata[idx]

    # Lookup the prospectus URL
    url = full_text_search(SEC_API_KEY, str(cusip))

    if url:
        # Navigate the already-open mini window using the saved handle (fallback to named lookup)
        display(Javascript(f"""
        (function(){{
          var href = {json.dumps(url)};
          var w = window.__miniFilingWin || window.open('', {json.dumps(WINDOW_NAME)});
          if (w && !w.closed) {{
            try {{ w.location.replace(href); }} catch(e) {{ try {{ w.location.href = href; }} catch(_) {{}} }}
            try {{ w.focus(); }} catch(e) {{}}
          }}
        }})();
        """))
    else:
        trace.figure.update_layout(annotations=[dict(
            text="No 424B2 filing found for this CUSIP (or API error).",
            xref="paper", yref="paper", x=0.5, y=1.08, showarrow=False, font=dict(color="#ffd166"),
        )])

    # Emphasize selected point
    new_sizes = state.orig_sizes.copy()
    new_sizes[idx] = state.orig_sizes[idx] * 2.25
    trace.marker.size = new_sizes

    x0, y0 = trace.x[idx], trace.y[idx]
    ax = -40 if status == "Alive" else 40
    ay = -40

    trace.figure.layout.annotations = ()
    trace.figure.add_annotation(
        x=x0, y=y0, xref="x", yref="y",
        text=(
            f"<b><a href='{url or '#'}' target='_blank' style='color:#37f3ff'>{cusip}</a></b><br>"
            f"<b>Start:</b> {start_s}<br>"
            f"<b>End:</b> {end_s}<br>"
            f"<b>Status:</b> {status}<br>"
            f"<b>Lifespan:</b> {float(lifespan):.2f} mo"
        ),
        showarrow=True, arrowhead=2, ax=ax, ay=ay,
        bgcolor="rgba(5,7,12,0.85)", bordercolor="#eaf2ff", borderwidth=1, font=dict(color="#eaf2ff"),
    )

# ────────────────────────────────────────────────────────────────────────────────
# Reset handler

def reset_view(_=None):
    status_dd.value = "All"
    toggle_avg.value = False
    if state.fw and len(state.fw.data):
        trace = state.fw.data[0]
        if state.orig_sizes:
            trace.marker.size = state.orig_sizes
        state.fw.layout.annotations = ()
        state.fw.update_xaxes(autorange=True)
        state.fw.update_yaxes(autorange=True)
        state.last_idx = None
    update_figure()

reset_btn.on_click(reset_view)

# ────────────────────────────────────────────────────────────────────────────────
# Wire up & render

status_dd.observe(update_figure, names="value")
toggle_avg.observe(update_figure, names="value")

controls = widgets.HBox([status_dd, toggle_avg, reset_btn], layout={"padding": "10px", "gap": "8px"})
update_figure()

ui = widgets.VBox([controls, output], layout={"align_items": "center"})
ui
